In [1]:
# merci PEP 526 !
_: print((list(map(_.append, map(chr, [104,101,108,108,111]))),
          "".join(_))[1]) = []

hello


# L'interpréteur Python, quel sale type

PyConFR 2017, Toulouse

par Serge « sans paille » Guelton

avec la bénédiction de QuarksLab

## Round 0

Quel type pour...

In [2]:
id  # id(obj: Any) -> int

<function id>

In [3]:
int  # int(obj: SupportsInt) -> int

int

In [4]:
list.append  # list.append(self: List[T], obj: T) -> None

<method 'append' of 'list' objects>

## Round 1

In [5]:
from typing import TypeVar  # PEP 484
T = TypeVar('T')

def add(self: T, other: T) -> T:  # PEP 3107
    return self + other

In [6]:
from typing import List, Tuple
t0: int = add(1, 2)
t1: List[int] = add([1], [2])
t2: Tuple = add((1,), (2,))

## Round 2

In [7]:
from typing import List

l: List[int] = [1,2,3]

def indexer(i):
    return l[i]

In [8]:
from typing import overload, Iterable
@overload
def indexer(i: int) -> int:
    pass
@overload
def indexer(i: slice) -> Iterable[int]:
    pass
def indexer(i):
    return l[i]

## Round 3

Quel type pour

In [9]:
def bar(x):
    return str(x)

def foo(x):
    return bar if int(x) else None

In [10]:
from typing import Optional, SupportsInt, Any, Callable

def bar(x: Any) -> str:
    return str(x)

def foo(x: SupportsInt) -> Optional[Callable[[Any], str]]:
    return bar if int(x) else None

In [11]:
from typing import Iterable, Tuple, List

x: Iterable[int] = range(3)
y: Iterable[int] = reversed(range(3))
l0: Iterable[Tuple[int, int]] = zip(x, y)
l1: Tuple[Iterable[int], Iterable[int]] = zip(*l0)

In [12]:
from random import randint
n = randint(1, 4)
t0: List[Tuple[int]] = [(1, )] * n
l2: Tuple[(int,) * n] = zip(*t0)

## K.O.

In [13]:
l: Any = eval("1 + 2")

In [14]:
%%file pyconfr2017/ko.py
a: int = 1

Overwriting pyconfr2017/ko.py


In [15]:
ko = __import__("pyconfr2017.ko")

## Pour avoir l'air savant

*typage nominal*

Utilise le nom du type des objets
    pour faire les calculs de type
    
*typage structurel*

Utilise les membres des objets
    pour faire des calculs de type
    

## Python

Python utilise le *duck typing*

Typage **structurel** à l'exécution

## Suis-je un fan de parkour ?

In [16]:
def isiterable0(x):  # nominal
    return isinstance(x, (set, tuple, list, dict, str))

def isiterable1(x):  # structurel
    return hasattr(x, '__iter__')

In [17]:
def isiterable2(x):  # duck type v0
    try:
        x.__iter__()
        return True
    except:
        return False

In [18]:
def isiterable3(x):  # duck type v1
    try:
        iter(x)
        return True
    except:
        return False

### Quizz

- Quelle différence entre ``isiterable1`` et ``isiterable2`` ?
- Quelle différence entre ``isiterable2`` et ``isiterable3`` ?

## Approche de haut niveau

In [19]:
from collections.abc import Iterable

def isiterable(x):
    return isinstance(x, Iterable)

In [20]:
def check_iterable(l):
    return isiterable0(l), isiterable1(l), isiterable2(l), isiterable3(l), isiterable(l)

l = [1, 2, 3, 5]
check_iterable(l)

(True, True, True, True, True)

## Peut-on se ``mock``er ?

In [21]:
class EmptySequence(object):
    
    def __iter__(self): yield
    def __len__(self): return 0
    
es = EmptySequence()

check_iterable(es)

(False, True, True, True, True)

## Test de robustesse n°0

In [22]:
class Infinity(object):
    def __getitem__(self, _):
        return 0
    
infnty = Infinity()
check_iterable(infnty)

(False, False, False, True, False)

## Test de robustesse n°1

In [23]:
class Hole(object):
    def __iter__(self, _):
        pass
    
h = Hole()
check_iterable(h)

(False, True, False, False, True)

## ``__subclasshook__``

In [24]:
import abc  # PEP 3119

class Appendable(abc.ABC):
    
    @classmethod
    def __subclasshook__(cls, C):
        return any('append' in B.__dict__ for B in C.mro())

    
class DevNull(object):
    
    def append(self, value):
        pass
    
    def __len__(self):
        return 0
    

In [25]:
issubclass(DevNull, Appendable)

True

In [26]:
class TraitsFactory(object):
    def __getitem__(self, method_names):
        if not isinstance(method_names, tuple):
            method_names = method_names,
            
        class SlotCheck(abc.ABC):
    
            @classmethod
            def __subclasshook__(cls, C):
                return all(any(method_name in B.__dict__ for B in C.mro())
                           for method_name in method_names)
        return SlotCheck
Members = TraitsFactory()

In [27]:
issubclass(DevNull, Members['append', '__len__'])

True

In [28]:
issubclass(DevNull, Members['append', 'clear'])

False

**NB** ça ne teste que les *noms* de membre, pas leur type...

## Contrat implicite

In [29]:
class Twice(list):
    def append(self, value, times):
        for _ in range(times):
            super(Twice, self).append(value)

tw = Twice()
tw.append("ore", 2)
len(tw)

2

In [30]:
isinstance(tw, Members['append', '__len__'])

True

In [31]:
tw.append("aison")  # gentlemna contract again

TypeError: append() missing 1 required positional argument: 'times'

## Le gadget de l'``inspect``eur

In [32]:
import inspect
inspect.signature(Twice.append).parameters

mappingproxy({'self': <Parameter "self">,
              'times': <Parameter "times">,
              'value': <Parameter "value">})

### Un bon moyen pour tester l'arité ?

In [33]:
inspect.signature(list.append)

ValueError: no signature found for builtin <method 'append' of 'list' objects>

In [34]:
from typing import Callable
isinstance(list.append, Callable)

True

In [35]:
from typing import List, TypeVar; T = TypeVar('T')
isinstance(list.append, Callable[[List[T], T], None])

TypeError: Parameterized generics cannot be used with class or instance checks

## ``import typeguard``

In [36]:
from typeguard import typechecked

In [37]:
def aff(x: int) -> int:
    return x * 2 + 1

In [38]:
aff(2)

5

In [39]:
aff("2")

TypeError: must be str, not int

In [40]:
taff = typechecked(aff)

In [41]:
taff(2)

5

In [42]:
taff("2")

TypeError: type of argument "x" must be int; got str instead

In [43]:
taff(2.)

TypeError: type of argument "x" must be int; got float instead

In [44]:
from numbers import Number  # PEP 3141
@typechecked
def taff(x: Number) -> Number:
    return x * 2 + 1

In [45]:
taff(1), taff(1.), taff(1j)

(3, 3.0, (1+2j))

In [46]:
print("** without type checking **")
%timeit aff(2)
print("** with type checking **")
%timeit taff(2)

** without type checking **
92.5 ns ± 2.53 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)
** with type checking **
31.5 µs ± 103 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


## Un peu plus loin

In [47]:
@typechecked
def index(x: Members["__getitem__"]) -> None:
    x[1]

In [48]:
index([1,2])

In [ ]:
@typechecked
def pouce(x: Members['__getitem__']) -> None:
    return x[0]
pouce([0])

In [ ]:
from typing import List, TypeVar; T = TypeVar('T')

@typechecked
def majeur(x: List[T]) -> T:
    return x[2]

In [ ]:
majeur([1,2,3])

In [ ]:
majeur([1, "1", 1])

In [ ]:
@typechecked
def step(x : List[T]) -> List[T]:
    return x + [x[-2] + x[-1]]

In [ ]:
step([1, 2])

In [ ]:
from functools import reduce
reduce(lambda x, _: step(x), range(10), [0, 1])

In [ ]:
from typing import Tuple
@typechecked
def step(x : Tuple) -> Tuple:
    return x + (x[-2] + x[-1],)
step((1,2))

In [ ]:
@typechecked
def step(x : Tuple) -> Tuple:
    return x + (x[-2] + x[-1],)
step((1,2))

## MyPy

In [ ]:
from mypy.api import run as mypy_runner
def mypy(*args):
    print( mypy_runner(args)[0])

In [ ]:
%%file pyconfr2017/mypy0.py
from typing import Tuple

def step(x : Tuple) -> Tuple:
    return x + (x[-2] + x[-1],)
step([1, 2])

In [ ]:
mypy("pyconfr2017/mypy0.py")

In [ ]:
%%file pyconfr2017/mypy1.py
from typing import Tuple

def step(x : Tuple) -> Tuple:
    return x + (x[-2] + x[-1],)
step((1, 2))

In [ ]:
mypy("pyconfr2017/mypy1.py")

In [ ]:
%%file pyconfr2017/mypy2.py
from typing import Tuple, Any

def step(x : Tuple[int, ...]) -> Tuple[int, ...]:
    return x + (x[-2] + x[-1],)
step((1, 2))

In [ ]:
mypy("pyconfr2017/mypy2.py")

## Python n'est il pas Dynamique?

In [ ]:
float_mode = True
scalar = float if float_mode else int

@typechecked
def div(n: scalar):
    return 2 / n

div(scalar(3))

In [ ]:
%%file pyconfr2017/mypy3.py
float_mode = True
scalar_type = float if float_mode else int
    
def div(n: scalar_type):
    return 2 / n

div(scalar_type(3))

In [ ]:
mypy("pyconfr2017/mypy3.py")

### Le coup fatal

In [ ]:
import numpy
help(numpy.sum)

# Bref, l'interpreteur Python, quel branquignol ?

- Python a été conçu comme un langage de glue
- Dynamisme au cœur du langage
- Utilisons le pour faire ce pour quoi il a été conçu !

** Bonus **

- Pour toi, ML n'est **pas** une catégorie de langage ?
- Tu aimes la recherche et l'ingénierie ?
- Ça recrute à l'École de Managment de Lyon sur un poste de *research engineer* ½ R&D ½ Prof